In [1]:
import sagemaker
from sklearn.model_selection import train_test_split
import boto3
import pandas as pd
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import os
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torch.optim as optim
from PIL import Image
from sklearn.model_selection import train_test_split
import optuna
from io import BytesIO

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sergioq2/.config/sagemaker/config.yaml


/mnt/c/AI/sagemaker/sm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
s3 = boto3.resource('s3')
bucket = s3.Bucket('traumamlcompetition')

In [33]:
names_1 = []
for _ in bucket.objects.filter(Prefix='images_1/augmented_images/'):
    name = _.key
    name = name.split('/')[2]
    names_1.append(name)

In [34]:
names_0 = []
for _ in bucket.objects.filter(Prefix='images_0/images_0/'):
    name = _.key
    name = name.split('/')[2]
    names_0.append(name)

In [75]:
def open_s3_image(s3_url):
    s3_bucket = s3_url.split('/')[2]
    s3_key = '/'.join(s3_url.split('/')[3:])
    s3_client = boto3.client('s3')
    response = s3_client.get_object(Bucket=s3_bucket, Key=s3_key)
    image_data = response['Body'].read()
    return image_data

In [81]:
directorio_imagenes_1 = "s3://traumamlcompetition/images_1/augmented_images/"
directorio_imagenes_0 = "s3://traumamlcompetition/images_0/images_0/"

transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),  
    transforms.Resize((128, 128)),  
    transforms.ToTensor(),
    transforms.Normalize(mean=0., std=1.) 
])

data = [(os.path.join(directorio_imagenes_1, filename), 1) for filename in names_1] + [(os.path.join(directorio_imagenes_0, filename), 0) for filename in names_0]

class CustomDataset(Dataset):
    def __init__(self, data, transform=None):
        self.data = data
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path, label = self.data[idx]
        
        img_data = open_s3_image(img_path)
        image = Image.open(BytesIO(img_data))

        if self.transform:
            image = self.transform(image)

        label = torch.tensor(label, dtype=torch.float32) 

        return image, label

data_train, data_test = train_test_split(data, test_size=0.15, random_state=42, stratify=[label for _, label in data])

train_dataset = CustomDataset(data_train, transform=transform)
test_dataset = CustomDataset(data_test, transform=transform)

In [3]:
sm_boto3 = boto3.client("sagemaker")
sess = sagemaker.Session()
region = sess.boto_session.region_name
bucket = 'traumamlcompetition'
role = sagemaker.get_execution_role()

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sergioq2/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sergioq2/.config/sagemaker/config.yaml


In [164]:
import csv

images = []
labels = []

for image, label in train_dataset:
    images.append(image.numpy())  # Convierte el tensor de imagen a un array numpy
    labels.append(label.numpy())  # Convierte el tensor de etiqueta a un array numpy

csv_filename = 'dataset.csv'
with open(csv_filename, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerows(zip(images, labels))

In [165]:
images = []
labels = []

for image, label in test_dataset:
    images.append(image.numpy()) 
    labels.append(label.numpy()) 

# Escribir los datos en un archivo CSV
csv_filename = 'dataset_test.csv'
with open(csv_filename, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerows(zip(images, labels))

In [4]:
sk_prefix = "sagemaker/datacontainer"
trainpath = sess.upload_data(
    path="dataset.csv", bucket=bucket, key_prefix=sk_prefix
)

testpath = sess.upload_data(
    path="dataset_test.csv", bucket=bucket, key_prefix=sk_prefix
)
print(trainpath)
print(testpath)

s3://traumamlcompetition/sagemaker/datacontainer/dataset.csv
s3://traumamlcompetition/sagemaker/datacontainer/dataset_test.csv


In [92]:
batch_size = 32
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))  # Normaliza los datos si es necesario
])

local_train_dataset = CustomDataset(train_dataset, transform=transform)
local_test_dataset = CustomDataset(test_dataset, transform=transform)

train_loader = DataLoader(local_train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(local_test_dataset, batch_size=batch_size, shuffle=False)

In [5]:
%%writefile script.py

import sagemaker
from sklearn.model_selection import train_test_split
import boto3
import pandas as pd
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import os
import pandas as pd
import numpy as np
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torch.optim as optim
from PIL import Image
from sklearn.model_selection import train_test_split
import optuna
from io import BytesIO

class Net(nn.Module):
    def __init__(self, trial, num_conv_layers, num_filters, num_neurons, drop_conv2, drop_fc1):
        super(Net, self).__init__()                                                     # Initialize parent class
        in_size = 264                                                                    # Input image size (28 pixels)
        kernel_size = 3                                                                 # Convolution filter size

        # Define the convolutional layers
        self.convs = nn.ModuleList([nn.Conv2d(1, num_filters[0], kernel_size=(3, 3))])  # List with the Conv layers
        out_size = in_size - kernel_size + 1                                            # Size of the output kernel
        out_size = int(out_size / 2)                                                    # Size after pooling
        for i in range(1, num_conv_layers):
            self.convs.append(nn.Conv2d(in_channels=num_filters[i-1], out_channels=num_filters[i], kernel_size=(3, 3)))
            out_size = out_size - kernel_size + 1                                       # Size of the output kernel
            out_size = int(out_size/2)                                                  # Size after pooling

        self.conv2_drop = nn.Dropout2d(p=drop_conv2)                                    # Dropout for conv2
        self.out_feature = num_filters[num_conv_layers-1] * out_size * out_size         # Size of flattened features
        self.fc1 = nn.Linear(self.out_feature, num_neurons)                             # Fully Connected layer 1
        self.fc2 = nn.Linear(num_neurons, 1)                                           # Fully Connected layer 2
        self.p1 = drop_fc1                                                              # Dropout ratio for FC1

        # Initialize weights with the He initialization
        for i in range(1, num_conv_layers):
            nn.init.kaiming_normal_(self.convs[i].weight, nonlinearity='relu')
            if self.convs[i].bias is not None:
                nn.init.constant_(self.convs[i].bias, 0)
        nn.init.kaiming_normal_(self.fc1.weight, nonlinearity='relu')

    def forward(self, x):
        for i, conv_i in enumerate(self.convs):  # For each convolutional layer
            if i == 2:  # Add dropout if layer 2
                x = F.relu(F.max_pool2d(self.conv2_drop(conv_i(x)), 2))  # Conv_i, dropout, max-pooling, RelU
            else:
                x = F.relu(F.max_pool2d(conv_i(x), 2))                   # Conv_i, max-pooling, RelU

        x = x.view(x.size(0), -1)                     # Flatten tensor
        x = F.relu(self.fc1(x))                              # FC1, RelU
        x = F.dropout(x, p=self.p1, training=self.training)  # Apply dropout after FC1 only when training
        x = self.fc2(x)

        return x           

criterio = nn.BCEWithLogitsLoss()

def train(model, optimizer):
    num_epochs = 16
    for epoch in range(num_epochs):
        model.train()
        total_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterio(outputs, labels.unsqueeze(1))
            loss.backward()
            optimizer.step()
            total_loss += loss.item()


def test(network):
    network.eval()         # Set the module in evaluation mode (only affects certain modules)
    correct = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = network(images)
            predicted = torch.round(torch.sigmoid(outputs))
            correct += (predicted == labels.unsqueeze(1)).sum().item()
    accuracy_test = correct / len(test_loader.dataset)

    return accuracy_test

def objective(trial):
    # Define range of values to be tested for the hyperparameters
    num_conv_layers = trial.suggest_int("num_conv_layers", 2, 3)  # Number of convolutional layers
    num_filters = [int(trial.suggest_discrete_uniform("num_filter_"+str(i), 16, 128, 16))
                   for i in range(num_conv_layers)]              # Number of filters for the convolutional layers
    num_neurons = trial.suggest_int("num_neurons", 10, 260, 10)  # Number of neurons of FC1 layer
    drop_conv2 = trial.suggest_float("drop_conv2", 0.2, 0.5)     # Dropout for convolutional layer 2
    drop_fc1 = trial.suggest_float("drop_fc1", 0.2, 0.5)         # Dropout for FC1 layer

    # Generate the model
    model = Net(trial, num_conv_layers, num_filters, num_neurons, drop_conv2,  drop_fc1).to(device)

    # Generate the optimizers
    optimizer_name = trial.suggest_categorical("optimizer", ["Adam", "RMSprop", "SGD"])  # Optimizers
    lr = trial.suggest_float("lr", 1e-5, 1e-1, log=True)                                 # Learning rates
    optimizer = getattr(optim, optimizer_name)(model.parameters(), lr=lr)

    # Training of the model
    for epoch in range(n_epochs):
        train(model, optimizer)  # Train the model
        accuracy = test(model)   # Evaluate the model

        # For pruning (stops trial early if not promising)
        trial.report(accuracy, epoch)
        # Handle pruning based on the intermediate value.
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()

    return accuracy


if __name__ == '__main__':

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # --- Parameters ----------------------------------------------------------
    n_epochs = 10                         # Number of training epochs
    batch_size_train = 64                 # Batch size for training data
    batch_size_test = 64               # Batch size for testing data
    number_of_trials = 60                # Number of Optuna trials
    limit_obs = True                      # Limit number of observations for faster computation


    if limit_obs:  # Limit number of observations
        number_of_train_examples = 5 * batch_size_train  # Max train observations
        number_of_test_examples = 2 * batch_size_test      # Max test observations
    else:
        number_of_train_examples = 60                   # Max train observations
        number_of_test_examples = 10                    # Max test observations
    # -------------------------------------------------------------------------

    # Make runs repeatable
    random_seed = 1
    torch.backends.cudnn.enabled = False  # Disable cuDNN use of nondeterministic algorithms
    torch.manual_seed(random_seed)

    # Create directory 'files', if it doesn't exist, to save the dataset
    directory_name = 'files'
    if not os.path.exists(directory_name):
        os.mkdir(directory_name)

    train_loader = DataLoader(train_dataset, batch_size=batch_size_train, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size_test, shuffle=False)

    # Create an Optuna study to maximize test accuracy
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=number_of_trials)

    # Find number of pruned and completed trials
    pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
    complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])

    # Display the study statistics
    print("\nStudy statistics: ")
    print("  Number of finished trials: ", len(study.trials))
    print("  Number of pruned trials: ", len(pruned_trials))
    print("  Number of complete trials: ", len(complete_trials))

    trial = study.best_trial
    print("Best trial:")
    print("  Value: ", trial.value)
    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

    # Save results to csv file
    df = study.trials_dataframe().drop(['datetime_start', 'datetime_complete', 'duration'], axis=1)  # Exclude columns
    df = df.loc[df['state'] == 'COMPLETE']        # Keep only results that did not prune
    df = df.drop('state', axis=1)                 # Exclude state column
    df = df.sort_values('value')                  # Sort based on accuracy
    df.to_csv('optuna_results.csv', index=False)  # Save to csv file

    # Display results in a dataframe
    print("\nOverall Results (ordered by accuracy):\n {}".format(df))

    # Find the most important hyperparameters
    most_important_parameters = optuna.importance.get_param_importances(study, target=None)

    # Display the most important hyperparameters
    print('\nMost important hyperparameters:')
    for key, value in most_important_parameters.items():
        print('  {}:{}{:.2f}%'.format(key, (15-len(key))*' ', value*100))

Writing script.py


In [5]:
from sagemaker.pytorch import PyTorch

pt_estimator = PyTorch(
    entry_point="script.py",
    role='arn:aws:iam::646932767172:role/service-role/AmazonSageMaker-ExecutionRole-20230923T203035',
    framework_version="1.4.0",
    py_version="py3",
    instance_count=2,
    output_path='s3://traumamlcompetition/',
    sagemaker_session=sagemaker.Session(),
    instance_type="ml.c5.xlarge"
)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sergioq2/.config/sagemaker/config.yaml


In [6]:
pt_estimator.fit({"train": trainpath, "test": testpath}, wait=True)

Using provided s3_resource


INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating training-job with name: pytorch-training-2023-10-02-17-58-26-404


2023-10-02 17:58:28 Starting - Starting the training job...
2023-10-02 17:58:43 Starting - Preparing the instances for training......
2023-10-02 17:59:47 Downloading - Downloading input data...
2023-10-02 18:00:22 Training - Downloading the training image..bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2023-10-02 18:00:42,662 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2023-10-02 18:00:42,666 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2023-10-02 18:00:42,678 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2023-10-02 18:00:42,680 sagemaker_pytorch_container.training INFO     Invoking user training script.
2023-10-02 18:00:42,838 sagemaker-containers INFO     Module default_user_module_name does not provide a setup.py. 
Generating setup.py
2023-10-02 18:00:42,838 sagemaker-containers INFO     Generating 

UnexpectedStatusException: Error for Training job pytorch-training-2023-10-02-17-58-26-404: Failed. Reason: AlgorithmError: framework error: 
Traceback (most recent call last):
  File "/opt/conda/lib/python3.6/site-packages/sagemaker_training/trainer.py", line 85, in train
    entrypoint()
  File "/opt/conda/lib/python3.6/site-packages/sagemaker_pytorch_container/training.py", line 99, in main
    train(framework.training_env())
  File "/opt/conda/lib/python3.6/site-packages/sagemaker_pytorch_container/training.py", line 60, in train
    six.reraise(info[0], err, info[2])
  File "/opt/conda/lib/python3.6/site-packages/six.py", line 703, in reraise
    raise value
  File "/opt/conda/lib/python3.6/site-packages/sagemaker_pytorch_container/training.py", line 53, in train
    capture_error=True, runner=framework.runner.ProcessRunnerType)
  File "/opt/conda/lib/python3.6/site-packages/sagemaker_containers/entry_point.py", line 100, in run
    wait, capture_error
  File "/opt/conda/lib/pytho